In [22]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import HBox, VBox, widgets


data = pd.read_csv("design_style_grid_likert3.csv")
styles = data["style"].tolist()
imp = data["style"].isin(["Modern", "Transitional", "Traditional"]).to_numpy()
# imp = data.important.to_numpy()
all_spectrums = [
    "Industrial--Rustic",
    "Formal--Informal",
    "Traditional--Modern",
    "Maximalist--Minimalist",
]
starting_spectrums = [
    "Industrial--Rustic",
    "Formal--Informal",
    "Traditional--Modern",
    "Maximalist--Minimalist",
]
midpoint = 3

# Create a Plotly figure
fig = go.FigureWidget()

def apply_forces_with_xy_barriers(points, barrier_limits, iterations=50, repulsion=0.005, damping=0.05):
    """
    Apply forces to adjust points' positions, including repulsive forces from barriers on both x and y axes.
    
    :param points: Initial positions of the points as a Numpy array of shape (n_points, 2).
    :param barrier_limits: A tuple of limits for the barriers on the axes: (x_min, x_max, y_min, y_max).
    :param barrier_force_distance: The maximum distance from the barriers at which the force is applied.
    :param barrier_repulsion: Repulsion factor for the barriers.
    :param iterations: Number of iterations to adjust the points.
    :param repulsion: Repulsion factor for the force calculation between points.
    :param damping: Damping factor to stabilize the system.
    :return: Adjusted positions of the points.
    """
    x_min, x_max, y_min, y_max = barrier_limits

    jit = 0.05
    points = points + np.random.uniform(low=-jit, high=jit, size=points.shape)

    n_points = points.shape[0]
    for _ in range(iterations):
        forces = np.zeros_like(points)
        for i in range(n_points):
            for j in range(i + 1, n_points):
                # Calculate repulsive force
                diff = points[i, :] - points[j, :]
                dist_squared = np.dot(diff, diff)
                if dist_squared > 0:
                    force_magnitude = repulsion / dist_squared
                    force = diff * force_magnitude
                    forces[i, :] += force
                    forces[j, :] -= force

            # Barrier forces
            barrier_force_distance = np.mean([barrier_limits[1] - barrier_limits[0], barrier_limits[3] - barrier_limits[2]]) / 4
            print(barrier_force_distance)
            for axis in range(2):  # 0 for x, 1 for y
                for barrier in (barrier_limits[axis * 2], barrier_limits[axis * 2 + 1]):
                    diff = points[i, axis] - barrier
                    if abs(diff) < barrier_force_distance:
                        barrier_force_magnitude = repulsion * (1 - abs(diff) / barrier_force_distance)
                        forces[i, axis] += barrier_force_magnitude * np.sign(diff)
                
        # Apply forces to points with damping
        points += forces * damping
        points[:, 0] = np.clip(points[:, 0], x_min, x_max)
        points[:, 1] = np.clip(points[:, 1], y_min, y_max)

    return points


# Function to update the plot
def update_point(vals):
    usr_pnt = np.array(vals).reshape([1, -1])
    usr_pnt = (usr_pnt - midpoint) / s_std
    usr_pnt = usr_pnt @ eigenvectors[:, :2]
    with fig.batch_update():
        fig.data[0].x = [usr_pnt[0, 0]]
        fig.data[0].y = [usr_pnt[0, 1]]

    # Stretch plot if outside range
    curr_max = max(max_val, np.abs(usr_pnt).max() + buf)
    fig.update_layout(
        xaxis={"range": [-curr_max, curr_max]},
        yaxis={"range": [-curr_max, curr_max + 2 * buf]},
    )


def initial_plot():
    global eigenvectors
    global max_val
    global buf
    global s_std
    global spectrum_names
    global disc1
    global disc2
    # global sliders
    global vboxes
    spectrum_names = [cb.description for cb in cboxes if cb.value]

    # Start by creating sliders
    vboxes = []
    for spec in all_spectrums:
        # Parse spectrum name
        s1, s2 = spec.split("--")

        # Create slider
        slider = widgets.FloatSlider(
            value=midpoint,  # Initial value
            min=1,  # Minimum value
            max=5,  # Maximum value
            step=1,
            description="",
            disabled=False,
            continuous_update=True,
            orientation="horizontal",
            readout=False,  # Hide the numeric value
            layout=widgets.Layout(width="300px"),  # Adjust the width as needed
        )

        # Add slider to visual
        vboxes.append(
            HBox(
                [
                    widgets.Label(
                        s1,
                        layout=widgets.Layout(
                            width="100px",
                            display="flex",
                            justify_content="flex-end",
                        ),
                    ),
                    slider,
                    widgets.Label(s2),
                ],
                layout={"display": "" if spec in spectrum_names else "none"},
            )
        )
        # sliders = [row[1] for row in vboxes]

    # Create disclaimers
    disc1 = widgets.Label("Note: This tool is for exploration only and is not exact.")
    disc2 = widgets.Label()

    X_df = data.reindex(columns=spectrum_names)
    X = X_df.to_numpy()

    # s_means = X.mean(axis=0)
    s_std = X.std(axis=0)
    X_norm = (X - midpoint) / s_std
    if len(spectrum_names) > 2:
        cov = X_norm.T @ X_norm
        eigenvalues, eigenvectors = np.linalg.eigh(cov)
        eigenvalues, eigenvectors = np.flip(eigenvalues), np.flip(eigenvectors, axis=1)
        disc2.value = (
            "Roughly"
            f" {eigenvalues[:2].sum() / eigenvalues.sum():.0%} of the information"
            f" from the {len(spectrum_names)} spectrums is preserved in this"
            " 2D visual."
        )
    elif len(spectrum_names) == 2:
        eigenvectors = np.identity(2)
        eigenvalues = np.ones(2)
        disc2.value = ""
    else:
        raise ValueError("Must have at least 2 spectrum names")

    X_trans = X_norm @ eigenvectors[:, :2]

    # Add noise for plotting
    jit = 0.05
    # X_trans = X_trans + np.random.uniform(low=-jit, high=jit, size=X_trans.shape)

    # Add scatter plot for the special point
    user_point = fig.add_scatter(
        x=[0],
        y=[0],
        mode="markers",
        marker=dict(size=60, color="#008080"),
        text=["Your Style"],
        hoverinfo="text",
    )

    # Add scatter plot for normal points
    colors = ["#a2d2ff", "#ffb5a7", "#bde0fe", "#c3bef0"]
    color_sq = [colors[i % len(colors)] for i in range(len(X_trans))]
    # adjusted_points = apply_forces(X_trans)
    barriers = (X_trans[:, 0].min(), X_trans[:, 0].max(), X_trans[:, 1].min(), X_trans[:, 1].max())
    adjusted_points = apply_forces_with_xy_barriers(X_trans, barrier_limits=barriers)
    fig.add_scatter(
        x=adjusted_points[:, 0],
        y=adjusted_points[:, 1],
        mode="markers",
        marker=dict(size=30, color=color_sq),
        text=styles,
        hoverinfo="text",
    )

    # Get plot bounds
    max_val = np.abs(adjusted_points).max()
    buf = 0.05 * max_val
    max_val += buf

    # Customize layout
    fig.update_layout(
        autosize=True,
        width=1200,
        height=600,
        showlegend=False,
        plot_bgcolor="#f5f5f5",  # Transparent background
        paper_bgcolor="#f5f5f5",  # Transparent paper
        xaxis={"visible": False, "range": [-max_val, max_val]},  # Hide the x-axis
        yaxis={
            "visible": False,
            "range": [-max_val, max_val + 2 * buf],
        },  # Hide the y-axis
        margin={"l": 0, "r": 0, "t": 0, "b": 0},  # Reduce margins
        title={
            "text": "Discover Your Style",
            "y": 0.95,
            "x": 0.5,
            "xanchor": "center",
            "yanchor": "top",
            "font": {
                "family": "Helvetica",
                "size": 35,
            },
        },
    )


def replot(b):
    # Declare use of global variables
    global eigenvectors
    global max_val
    global buf
    global s_std

    spectrum_names = [cb.description for cb in cboxes if cb.value]

    # Update which sliders are visible
    for i, hb in enumerate(vboxes):
        hb.layout.display = "" if all_spectrums[i] in spectrum_names else "none"

    # Redo PCA
    X_df = data.reindex(columns=spectrum_names)
    X = X_df.to_numpy()

    # s_means = X.mean(axis=0)
    s_std = X.std(axis=0)
    X_norm = (X - midpoint) / s_std
    if len(spectrum_names) > 2:
        cov = X_norm.T @ X_norm
        eigenvalues, eigenvectors = np.linalg.eigh(cov)
        eigenvalues, eigenvectors = np.flip(eigenvalues), np.flip(eigenvectors, axis=1)
        disc2.value = (
            "Roughly"
            f" {eigenvalues[:2].sum() / eigenvalues.sum():.0%} of the information"
            f" from the {len(spectrum_names)} spectrums is preserved in this"
            " 2D visual."
        )
    elif len(spectrum_names) == 2:
        eigenvectors = np.identity(2)
        eigenvalues = np.ones(2)
        disc2.value = ""
    else:
        raise ValueError("Must have at least 2 spectrum names")

    X_trans = X_norm @ eigenvectors[:, :2]

    # Add noise for plotting
    jit = 0.05
    # X_trans = X_trans + np.random.uniform(low=-jit, high=jit, size=X_trans.shape)

    # Add scatter plot for the special point
    update_point([hb.children[1].value for hb in vboxes if hb.layout.display == ""])

    # Re-adjust
    # adjusted_points = apply_forces(X_trans)
    barriers = (X_trans[:, 0].min(), X_trans[:, 0].max(), X_trans[:, 1].min(), X_trans[:, 1].max())
    adjusted_points = apply_forces_with_xy_barriers(X_trans, barrier_limits=barriers)

    # Update scatter plots
    with fig.batch_update():
        fig.data[1].x = adjusted_points[:, 0]
        fig.data[1].y = adjusted_points[:, 1]

    # Get plot bounds
    max_val = np.abs(adjusted_points).max()
    buf = 0.05 * max_val
    max_val += buf

    # Customize layout
    fig.update_layout(
        xaxis={"range": [-max_val, max_val]},  # Hide the x-axis
        yaxis={
            "range": [-max_val, max_val + 2 * buf],
        },
    )


# Create Checkboxes
cboxes = [
    widgets.Checkbox(
        value=opt in starting_spectrums,
        description=opt,
        style={"description_width": "initial"},
    )
    for opt in all_spectrums
]
confirm = widgets.Button(description="Confirm Selection")

# Attach the event handler to the button
confirm.on_click(replot)

# Generate initial plot
initial_plot()


# Observe function to wrap update
def on_slider_change(change):
    update_point([hb.children[1].value for hb in vboxes if hb.layout.display == ""])


# Link sliders to observe function
for hb in vboxes:
    hb.children[1].observe(on_slider_change, names="value")

# Create the widgets
ui = VBox(
    vboxes
    + [VBox(cboxes + [confirm])]
    + [HBox([VBox([widgets.Label(), disc1, disc2])])]
)

# Display the interactive plot with sliders
display(fig, ui)

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': '#008080', 'size': 60},
              'mode': 'markers',
              'text': [Your Style],
              'type': 'scatter',
              'uid': 'd33c42c7-5ad8-49c2-87ee-b9bf63d0d2aa',
              'x': [0],
              'y': [0]},
             {'hoverinfo': 'text',
              'marker': {'color': [#a2d2ff, #ffb5a7, #bde0fe, #c3bef0, #a2d2ff,
                                   #ffb5a7, #bde0fe, #c3bef0, #a2d2ff, #ffb5a7,
                                   #bde0fe, #c3bef0, #a2d2ff, #ffb5a7, #bde0fe,
                                   #c3bef0, #a2d2ff, #ffb5a7, #bde0fe, #c3bef0,
                                   #a2d2ff, #ffb5a7, #bde0fe, #c3bef0, #a2d2ff,
                                   #ffb5a7, #bde0fe, #c3bef0, #a2d2ff, #ffb5a7,
                                   #bde0fe, #c3bef0, #a2d2ff],
                         'size': 30},
              'mode': 'markers',
              'text': [Art Deco